## Amazon Bedrock Knowledge Bases - Retrieve API로 Q&A 애플리케이션 구축

### 컨텍스트

이 노트북에서는 Amazon Bedrock Knowledge Bases의 Retrieve API를 사용해 Q&A 애플리케이션을 구축하는 방법을 자세히 살펴봅니다. 지식 베이스에서 유사도 검색을 기반으로 원하는 수만큼 문서 청크를 가져오고, 관련 문서를 프롬프트와 함께 Anthropic Claude V2에 전달해 응답을 생성합니다.

Knowledge Base를 사용하면 Amazon Bedrock의 Foundation Model(FM)을 기업 데이터에 안전하게 연결해 Retrieval Augmented Generation(RAG)을 구현할 수 있습니다. 추가 데이터에 접근하면 모델이 더 관련성 있고 컨텍스트에 특화된 정확한 응답을 생성하면서 FM을 지속적으로 재학습할 필요가 없습니다. Knowledge Base에서 검색된 모든 정보에는 출처가 제공되어 투명성을 높이고 환각을 줄입니다. 콘솔에서 Knowledge Base를 만드는 방법은 [문서](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html)를 참고하세요.
노트북은 두 부분으로 구성됩니다:
- 1부에서는 Amazon Bedrock의 Foundation Model과 함께 `RetrieveAPI`를 사용하는 방법을 설명합니다. 여기서는 `amazon.nova-pro-v1:0` 모델을 사용합니다. 
- 2부에서는 LangChain 통합을 소개합니다.

### 패턴

이 솔루션은 Retrieval Augmented Generation(RAG) 패턴으로 구현할 수 있습니다. RAG는 언어 모델 외부의 데이터(비모수 데이터)를 검색해 관련 데이터를 프롬프트에 추가합니다. 여기서는 콘솔/SDK로 생성한 Knowledge Base에서 효과적으로 RAG를 수행합니다. 

### 사전 준비

질문에 답하려면 문서를 처리해 Knowledge Base에 저장해야 합니다. 이 노트북에서는 Amazon Bedrock Knowledge Bases를 생성하기 위해 `synthetic dataset for 10K financial reports`를 사용합니다. 

1. 문서(데이터 소스)를 Amazon S3 버킷에 업로드합니다.
2. Amazon Bedrock Knowledge Bases는 [01_create_ingest_documents_test_kb_multi_ds.ipynb](/knowledge-bases/01-rag-concepts/01_create_ingest_documents_test_kb_multi_ds.ipynb)를 사용해 생성합니다.
3. Knowledge Base ID를 기록해 둡니다.

<!-- ![data_ingestion](./images/data_ingestion.png) -->
<img src="./images/data_ingestion.png" width=50% height=20% />

#### 노트북 워크스루

이 노트북에서는 Amazon Bedrock Knowledge Bases가 제공하는 `Retrieve API`를 사용합니다. 이 API는 사용자 질의를 임베딩으로 변환하고 Knowledge Base를 검색하며, 의미 기반 검색 결과 위에 사용자 정의 워크플로를 구축할 수 있도록 관련 결과를 반환합니다. `Retrieve API`의 출력에는 `검색된 텍스트 청크`, 소스 데이터의 `위치 유형`과 `URI`, 검색의 관련성 `점수`가 포함됩니다.

생성된 텍스트 청크를 원래 프롬프트와 결합해 `amazon.nova-pro-v1:0` 모델에 전달하고, 사용 사례에 맞는 프롬프트 엔지니어링 패턴으로 응답을 생성합니다.

### USE CASE

#### 데이터 세트

이 예제에서는 Octank의 재무 10-K 보고서(합성 데이터 세트)를 텍스트 코퍼스로 사용해 Q&A를 수행합니다. 이 데이터는 이미 Amazon Bedrock Knowledge Bases에 적재되어 있으며, 실행하려면 `knowledge base id`가 필요합니다.
실제 사용 사례에서는 도메인별 주제에 맞춰 다양한 파일을 동기화한 뒤 Knowledge Bases의 Retrieve API를 사용해 동일한 방식으로 모델 응답을 평가할 수 있습니다.

### Python 3.10

⚠ 이 실습은 Python 3.10 런타임에서 실행해야 합니다. ⚠

Amazon SageMaker Studio 외부의 로컬 환경에서 워크숍을 진행하는 경우 Python 런타임이 3.10 이상인지 확인하세요.

### 설정

이 노트북을 실행하려면 다음 패키지를 설치해야 합니다.

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

#### 위 종속성으로 설치한 패키지를 적용하려면 커널을 재시작하세요

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
%store -r kb_id
# kb_id = "<knowledge base id>" If you have already created knowledge base, comment the `store -r kb_id` and provide knowledge base id here.

### Bedrock 클라이언트를 초기화하려면 아래 단계를 따르세요:

1. 필요한 라이브러리를 임포트합니다. 여기에는 Bedrock 모델 선택을 위한 LangChain, LLM과 임베딩 모델 인스턴스를 보관하는 LlamaIndex 서비스 컨텍스트가 포함됩니다. 서비스 컨텍스트는 이후 Q&A 애플리케이션의 응답을 평가할 때 사용합니다. 

2. Knowledge Base에서 `retrieve` API로 모든 텍스트 청크를 검색한 뒤 RAG 패턴으로 질의 완성을 수행하기 위해 `amazon.nova-pro-v1:0`을 대형 언어 모델로 초기화합니다.

In [ ]:
import boto3
import pprint
from botocore.client import Config
import json

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

### 1부 - Amazon Bedrock Foundation Model과 함께 Retrieve API 사용

사용자 질의를 임베딩으로 변환하고 Knowledge Base를 검색하며 관련 결과를 반환해 의미 기반 검색 결과 위에 사용자 정의 워크플로를 구성할 수 있도록 하는 Amazon Bedrock Knowledge Bases의 `Retrieve API`를 호출하는 함수를 정의합니다. 

Retrieve API는 Amazon Bedrock이 제공하는 어떤 Foundation Model이든 사용할 수 있으며, 사용 사례에 따라 HYBRID 또는 SEMANTIC 검색 유형을 선택할 수 있는 유연성을 제공합니다. 
Hybrid Search 기능에 대한 자세한 내용은 [블로그](#https://aws.amazon.com/blogs/machine-learning/knowledge-bases-for-amazon-bedrock-now-supports-hybrid-search/)를 참고하세요.

In [5]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

#### LLM 응답을 조회하기 전에 Knowledge Base ID를 초기화하세요

이제 `retrieve API`를 호출하면서 `knowledge base id`, `number of results`, `query`를 파라미터로 전달합니다. 

`score`: 반환된 각 텍스트 청크의 점수를 확인하면 질의와 얼마나 밀접하게 일치하는지 상관성을 파악할 수 있습니다.

In [ ]:
query = "What was the total operating lease liabilities and total sublease income of the Octank as of December 31, 2022?"
response = retrieve(query, kb_id, 5)
retrievalResults = response['retrievalResults']
pp.pprint(retrievalResults)

### retrieveAPI 응답에서 텍스트 청크 추출

아래 셀에서는 검색 결과에서 컨텍스트를 가져옵니다.

In [7]:
# fetch context from the response
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [ ]:
contexts = get_contexts(retrievalResults)
pp.pprint(contexts)

### 모델에 특화된 프롬프트로 응답을 개인화하기

아래 프롬프트는 모델이 금융 자문 AI 시스템처럼 동작해 가능한 경우 사실 기반 및 통계 정보를 활용해 질문에 답하도록 안내합니다. 앞서 받은 `Retrieve API` 응답을 프롬프트의 `{contexts}`에 포함해 모델이 참고하도록 하고, 사용자 `query`도 함께 전달합니다.  

In [9]:
prompt = f"""
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

### Amazon Bedrock의 Foundation Model 호출
이 예제에서는 Amazon Bedrock의 `amazon.nova-pro-v1:0` Foundation Model을 사용합니다. 
- 경쟁사 대비 낮은 비용으로 높은 효용을 제공하도록 설계된 모델로, 대규모 AI 배포에 적합한 안정적인 고성능 모델입니다. Amazon Nova Pro는 이미지를 처리해 텍스트 출력을 반환할 수 있고, 300K 컨텍스트 윈도우를 제공합니다.
- 모델 특성
    - Image to text & code, 다국어 대화, 복잡한 추론 및 분석

In [ ]:
# payload with model paramters
messages=[{ "role":'user', "content":[{'text': prompt.format(contexts, query)}]}]
inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

additionalModelRequestFields = {
    "inferenceConfig": {
         "topK": 20
    }
}

In [ ]:
modelId = 'us.amazon.nova-pro-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
model_response = bedrock_client.converse(
    modelId=modelId, 
    messages=messages, 
    inferenceConfig=inf_params,
    additionalModelRequestFields=additionalModelRequestFields
)
print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

print("\n[Response Content Text]")
print(model_response["output"]["message"]["content"][0]["text"])

## Part 2 - LangChain 통합
이 노트북의 두 번째 부분에서는 Amazon Bedrock Knowledge Bases가 제공하는 Retrieve API와 LangChain을 함께 사용해 Q&A 애플리케이션을 구성합니다. 유사도 검색을 기반으로 원하는 수의 문서 청크를 가져오고, 이를 LangChain retriever와 통합한 뒤 `Amazon Nova Pro` 모델로 질문에 답변합니다.

In [ ]:
# from langchain.llms.bedrock import Bedrock
import langchain
from langchain_aws import ChatBedrock
from langchain_aws.retrievers import AmazonKnowledgeBasesRetriever

llm = ChatBedrock(model_id=modelId, 
                  client=bedrock_client)

LangChain에서 `AmazonKnowledgeBasesRetriever` 객체를 생성하면 Amazon Bedrock Knowledge Bases의 `Retrieve API`를 호출해 사용자 질의를 임베딩으로 변환하고 Knowledge Base를 검색한 뒤 관련 결과를 반환합니다. 이를 통해 의미 기반 검색 결과 위에 사용자 정의 워크플로를 더 세밀하게 구성할 수 있습니다. `Retrieve API` 출력에는 `retrieved text chunks`, 소스 데이터의 `location type`과 `URI`, 검색 관련성 `scores`가 포함됩니다.

In [ ]:
query = "What was the total operating lease liabilities and total sublease income of the Octank as of December 31, 2022?"
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": 
                          {"numberOfResults": 4,
                           'overrideSearchType': "SEMANTIC", # optional
                           }
                          },
        # endpoint_url=endpoint_url,
        # region_name=region,
        # credentials_profile_name="<profile_name>",
    )
docs = retriever.invoke(query)
pp.pprint(docs)

## 모델에 특화된 프롬프트로 응답을 개인화하기
아래 프롬프트는 모델이 금융 자문 AI 시스템처럼 동작해 가능한 경우 사실 기반 및 통계 정보를 활용해 질문에 답하도록 안내합니다. 위에서 받은 Retrieve API 응답을 프롬프트의 `{context}`에 포함해 모델이 참고하도록 하고, 사용자 `query`도 함께 전달합니다.

In [ ]:
from langchain_core.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""
claude_prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                               input_variables=["context","question"])

### 위에서 정의한 retriever와 LLM을 RetrievalQA Chain으로 결합해 Q&A 애플리케이션을 구성합니다.

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": claude_prompt}
)

In [ ]:
answer = qa.invoke(query)
pp.pprint(answer)

## 결론
RAG 기반 애플리케이션을 사용자화할 때는 Bedrock의 `InvokeModel` API를 직접 사용하거나, LangChain의 `AmazonKnowledgeBaseRetriever`로 통합해 Retrieve API를 활용할 수 있습니다.
Retrieve API는 Amazon Bedrock에서 제공하는 어떤 Foundation Model도 선택할 수 있는 유연함과, 사용 사례에 맞춰 HYBRID 또는 SEMANTIC 검색 유형을 선택할 수 있는 옵션을 제공합니다. 
Hybrid Search 기능에 대한 자세한 내용은 [블로그](#https://aws.amazon.com/blogs/machine-learning/knowledge-bases-for-amazon-bedrock-now-supports-hybrid-search/)를 참고하세요.

<div class="alert alert-block alert-warning">
<b>참고:</b> 비용이 발생하지 않도록 KB, OSS 인덱스 및 관련 IAM 역할과 정책을 반드시 삭제하세요.
</div>